In [2]:
import os
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import tqdm
from tqdm import tqdm
from scipy.spatial import distance
import sys
import pandas as pd
import matplotlib.pyplot as plt
from random import sample
import matplotlib.pyplot as plt
import pandas as pd
import statistics
from statistics import mean, stdev
import math

from retrofitting_funcs import get_embs, get_retro_embs

#extract nouns, verbs and adjectives 
os.chdir('/nfs/gpusan01/gbihani/context_div/WSD_data/mydata/')

with open('nouns.pkl', 'rb') as f:
    nouns = pickle.load(f)
    
with open('verbs.pkl', 'rb') as f:
    verbs = pickle.load(f)
    
with open('adjectives.pkl', 'rb') as f:
    adjectives = pickle.load(f)

### Loading an embedding file for (model: 'bert', layer: 0, dataset: 'semeval15_13')

In [4]:
data= get_embs('semeval15_13', 0, 'bert') #get original embeddings
van = [((y[0][0], y[0][1], y[0][2]), y[1]) for x in data for y in x]
van_vec = [x[1] for x in van]
vans_ = van

##final file to be used is vans_filt
vans_filt_ = [x for x in vans_ if ((x[0][1] in [y1[0] for y1 in nouns]) or (x[0][1] in [y2[0] for y2 in verbs]) or
                                     (x[0][1] in [y3[0] for y3 in adjectives]))] 


### Embedding format

In [8]:
#vans_filt_[i] contains ((sentence_id, word, word_sense),(embedding))
#sentence_id = sentence from original corpus which contains word, and the corresponding sense

vans_filt_[0]

((1, 'document', 'document%1:10:00::'),
 tensor([ 0.1707,  0.1397, -0.7813,  0.1900, -0.0214, -0.2148,  0.2324, -1.0346,
          0.6158, -1.1655, -0.5458,  0.1694, -0.2444,  0.9020,  0.3099, -0.6317,
          0.3262,  0.7722,  0.0199,  0.0187,  0.4702, -1.0844, -0.4344, -0.1920,
          0.5877, -0.3990, -0.5780, -0.0775,  0.6177, -1.0305, -0.0022,  0.3859,
          0.7303, -0.7913,  0.1190, -0.1364,  0.0713,  0.4134, -1.2910,  0.9592,
          0.4427, -0.4876, -1.0003,  0.1014, -0.4867,  0.2853, -0.3453,  0.7880,
          0.5581,  0.5480,  0.2823, -0.0076, -0.9888,  0.8198,  0.1645, -0.1814,
         -0.1852,  0.3802,  0.6396, -0.8974,  0.9390,  0.8562,  0.0986, -0.1087,
         -0.2598, -0.6736, -0.0488, -0.9092,  0.6115,  0.1314,  0.2512,  1.0240,
         -0.3224, -0.1817,  0.2863, -1.4376,  0.7122,  0.8311,  0.1375,  0.3231,
          0.1430,  1.1465, -0.4424,  0.5618, -0.1055,  0.1053,  0.5605,  0.7176,
         -0.4838,  0.5024,  0.7018, -0.6256, -0.6824,  0.2438,  0.122


______________________________________________

### Loading embeddings for every model, layer and dataset

You can use the previous example to load embeddings for every model, layer and dataset, one by one.

Or, you can loop over each dataset, model and layer to get every embedding

### models
model_types = ['bert', 'gpt2', 'xlnet', 'electra']

### datasets
datasets = ['semeval15_13', 'senseval3task1', 'senseval2', 'semeval13_12', 'semeval07_7']


***Don't run the following script before making suggested changes!***

In [4]:
##suggested changes are in comments

data = {}
data_r = {}
van = {}
van_r = {}
words = {}
words_r = {}
van_vec = {}
van_vec_r = {}


for model_type in model_types: ##currently set to get data for every model sequentially
    for layer in range(13): ##currently set to get data for every model layer sequentially
        for data_num in datasets: ##currently set to get data for every data set sequentially
            data[data_num] = get_embs(data_num, layer, model_type)
            van[data_num] = [((y[0][0], y[0][1], y[0][2]), y[1]) for x in data[data_num] for y in x]
            van_vec[data_num] = [x[1] for x in van[data_num]]

        ################### ORIGINAL EMBEDDINGS ###################

        wfs_ = []
        vans_ = []
        for data_num in datasets:
            vans_i = van[data_num]
            vans_ = vans_ + vans_i
            
        
        ##original embeddings for given configuration
    
        vans_filt_ = [x for x in vans_ if ((x[0][1] in [y1[0] for y1 in nouns]) or (x[0][1] in [y2[0] for y2 in verbs]) or
                                     (x[0][1] in [y3[0] for y3 in adjectives]))]
        
        ################### RETROFITTED EMBEDDINGS ###################
    
        for data_num in datasets:
            data_r[data_num] = get_retro_embs(data_num, layer, model_type, 1)
            van_r[data_num] = [((x[0][0], x[0][1], x[0][2]), x[1]) for x in data_r[data_num]]
            van_vec[data_num] = [x[1] for x in van_r[data_num]]

        wfs_r_ = []
        vans_r_ = []
        for data_num in datasets:
            vans_i_r = van_r[data_num]
            vans_r_ = vans_r_ + vans_i_r
        
        ##retrofitted embeddings for given configuration
        vans_filt_r_ = [x for x in vans_r_ if ((x[0][1] in [y1[0] for y1 in nouns]) or (x[0][1] in [y2[0] for y2 in verbs]) or
                                     (x[0][1] in [y3[0] for y3 in adjectives]))]
    
        
        ######## ADD CODE TO SAVE/USE/TRANSFER FILES TO ANOTHER DESTINATION ##########
        ### files to be used; original embeddings: vans_filt_ ; retrofitted embeddings: vans_filt_r_ ###
    